# Module Refinement Example

Load necessary packages

In [29]:
import numpy as np
import pandas as pd
from ModuleRefinement import utils, ModuleLBG, center_algorithms
import plotly.express as px

from sklearn.model_selection import StratifiedKFold

from orthrus.core.dataset import DataSet as DS

from IPython.utils import io




Initialize variables

In [2]:

species = 'mouse'
dataset_name = 'salmonella_Liver'
fold = 1

out_file = f'modules/5fold/fold{fold}_{dataset_name}.pickle'

module_file = f'./{out_file}'
refined_modules_dir = f'./refined_modules/'

  
center_methods = [['flag_mean',1,1]]

center_method = center_methods[0][0]
data_dimension = center_methods[0][1]
center_dimension = center_methods[0][2]

module_paths = [module_file,
                './refined_modules/flag_mean_1_1/5fold/fold1_salmonella_Liver.pickle']
algorithms = ['WGCNA', 'LBG']
organism = 'mmusculus'

Load data

In [3]:
data_path = f'../data/{dataset_name}.csv'

class_data, unique_labels, data_all, labels_all = utils.load_data(data_path)

Compute WGCNA Modules (Takes approximately 10 minutes)

In [30]:
 #keep random state at 0 for reproducible results
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
skf.get_n_splits(data_all, labels_all)
fold_number = 0 
for train_index, test_index in skf.split(data_all, labels_all):
    if fold_number == fold:
        split_data = data_all.iloc[train_index]
        split_labels = labels_all.iloc[train_index]

        with io.capture_output() as captured:
            r_value = utils.wgcna_modules(split_data, species, out_file)

        print(f'MODULES COMPUTED! R value = {r_value}')
    fold_number +=1

Compute the refined modules.

In [5]:
utils.refined_modules(split_data, module_file, center_methods, refined_modules_dir)

Compute GO significance for each module. 
Store in go_results.

In [6]:
go_results = pd.DataFrame(columns = 
                                    ['Data Set', 'Algorithm', 'Central Prototype', 
                                    'Data Dimension', 'Center Dimension', 'Fold', 
                                    'Module Number', 'GO Significance'])


for module_path, algorithm in zip(module_paths, algorithms):
    
    print(f'GO for {algorithm} modules')

    the_modules, all_features = utils.load_modules(module_path)
    module_number = 0
    for module in the_modules.iterrows():
        module_genes = module[1].item()   
                        
        mod_sig = utils.module_significance(module_genes, organism)

        if algorithm == 'WGCNA':
            row = pd.DataFrame(columns = list(go_results.columns),
                    data = [[dataset_name, algorithm, '',
                                '', '', fold,
                                module_number, mod_sig]])
        else:
            row = pd.DataFrame(columns = list(go_results.columns),
                    data = [[dataset_name, algorithm, center_method,
                                data_dimension, center_dimension, fold,
                                module_number, mod_sig]])            


        go_results = pd.concat([go_results, row])
        
        module_number +=1

go_results.head()

GO for WGCNA modules
GO for LBG modules


,Data Set,Algorithm,Central Prototype,Data Dimension,Center Dimension,Fold,Module Number,GO Significance
0,salmonella_Liver,WGCNA,,,,1,0,5823.552345
0,salmonella_Liver,WGCNA,,,,1,1,125.314059
0,salmonella_Liver,WGCNA,,,,1,2,4401.762648
0,salmonella_Liver,WGCNA,,,,1,3,162.896856
0,salmonella_Liver,WGCNA,,,,1,4,2175.596104


Compute the relative gain in GO significance.
Store in go_better.

In [7]:
go_better = pd.DataFrame(columns = ['Relative Gain', 'Fold', 'Prototype', 'Data Dimension', 'Center Dimension', 'Data Set'])

wgcna_query = (go_results["Algorithm"] == 'WGCNA') &\
              (go_results["Fold"] == fold) &\
              (go_results["Data Set"] == dataset_name)
if np.sum(wgcna_query) > 0:
    wgcna_total = go_results[wgcna_query]["GO Significance"].sum()
    eg_query = (go_results["Algorithm"] == 'LBG') &\
            (go_results["Central Prototype"] == center_method) &\
            (go_results["Data Dimension"] == data_dimension) &\
            (go_results["Center Dimension"] == center_dimension) &\
            (go_results["Fold"] == fold) &\
            (go_results["Data Set"] == dataset_name)                           
    eigengene_total = go_results[eg_query]["GO Significance"].sum()
    if np.sum(eg_query) > 0:
        relative_gain = (eigengene_total/(wgcna_total+.0001))-1
        row = pd.DataFrame(columns = go_better.columns,
                        data = [[relative_gain, fold, center_method, data_dimension, center_dimension, dataset_name]])
        go_better = go_better.append(row, ignore_index = True)


Compute SVM BSR for each module. 
Store in svm_results.

In [8]:
ds = DS(data = data_all, metadata = labels_all)

svm_results = pd.DataFrame(columns = 
                                ['Data Set', 'Algorithm', 'Central Prototype', 
                                'Data Dimension', 'Center Dimension', 'Fold', 
                                'Module Number', 'BSR', 'Size'])

for module_path, algorithm in zip(module_paths, algorithms):
    
    print(f'SVM for {algorithm} modules')

    the_modules, all_features = utils.load_modules(module_path)

    module_number = 0
    for module in the_modules.iterrows():
        mod_sig = 0
        module_genes = module[1].item()   

        module_size = len(list(module_genes))

        slice_dataset = ds.slice_dataset(feature_ids=np.array(module_genes))

        try:
            bsr = utils.loso_test(slice_dataset, fold)
        except ValueError:
            bsr = np.nan   

        if algorithm == 'WGCNA':
            row = pd.DataFrame(columns = list(svm_results.columns),
                        data = [[dataset_name, algorithm, np.nan,
                                    np.nan, np.nan, fold,
                                    module_number, bsr, module_size]])
        else:

            row = pd.DataFrame(columns = list(svm_results.columns),
                        data = [[dataset_name, algorithm, center_method,
                                    data_dimension, center_dimension, fold,
                                    module_number, bsr, module_size]])

        svm_results = svm_results.append(row, ignore_index = True)

        module_number +=1

svm_results.head()

SVM for WGCNA modules
SVM for LBG modules


,Data Set,Algorithm,Central Prototype,Data Dimension,Center Dimension,Fold,Module Number,BSR,Size
0,salmonella_Liver,WGCNA,NaN,NaN,NaN,1,0,1.0,7199
1,salmonella_Liver,WGCNA,NaN,NaN,NaN,1,1,0.5,201
2,salmonella_Liver,WGCNA,NaN,NaN,NaN,1,2,1.0,1095
3,salmonella_Liver,WGCNA,NaN,NaN,NaN,1,3,1.0,80
4,salmonella_Liver,WGCNA,NaN,NaN,NaN,1,4,1.0,1492


Compute the relative gain in BSR.
Store in svm_better.

In [9]:
svm_better = pd.DataFrame(columns = ['Relative Gain', 'Fold', 'Prototype', 'Data Dimension', 'Center Dimension', 'Data Set'])

wgcna_query = (svm_results["Algorithm"] == 'WGCNA') &\
              (svm_results["Fold"] == fold) &\
              (svm_results["Data Set"] == dataset_name)
if np.sum(wgcna_query) > 0:
    wgcna_total = svm_results[wgcna_query]["BSR"].sum()
    eg_query = (svm_results["Algorithm"] == 'LBG') &\
            (svm_results["Central Prototype"] == center_method) &\
            (svm_results["Data Dimension"] == data_dimension) &\
            (svm_results["Center Dimension"] == center_dimension) &\
            (svm_results["Fold"] == fold) &\
            (svm_results["Data Set"] == dataset_name)                           
    eigengene_total = svm_results[eg_query]["BSR"].sum()
    if np.sum(eg_query) > 0:
        relative_gain = (eigengene_total/(wgcna_total+.0001))-1
        row = pd.DataFrame(columns = svm_better.columns,
                        data = [[relative_gain, fold, center_method, data_dimension, center_dimension, dataset_name]])
        svm_better = svm_better.append(row, ignore_index = True)



In [10]:
svm_better

,Relative Gain,Fold,Prototype,Data Dimension,Center Dimension,Data Set
0,0.057684,1,flag_mean,1,1,salmonella_Liver


In [24]:
px.box(svm_results, x = 'Algorithm', y = 'Size', color = 'Algorithm', points= 'all', title = 'Module Sizes')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'hovertemplate': 'Algorithm=%{x}<br>Size=%{y}<extra></extra>',
              'legendgroup': 'WGCNA',
              'marker': {'color': '#636efa'},
              'name': 'WGCNA',
              'notched': False,
              'offsetgroup': 'WGCNA',
              'orientation': 'v',
              'showlegend': True,
              'type': 'box',
              'x': array(['WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA',
                          'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA'],
                         dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([7199, 201, 1095, 80, 1492, 197, 554, 411, 357, 122, 57, 71, 227, 379,
                          468, 252], dtype=object),
              'y0': ' ',
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'boxpoints': 'all',
              'hovertemplate': 'Algorithm=%{x}<br>Size=%{y}<extra></extra>',
              'legendgroup': 'LBG',
              'marker': {'color': '#EF553B'},
              'name': 'LBG',
              'notched': False,
              'offsetgroup': 'LBG',
              'orientation': 'v',
              'showlegend': True,
              'type': 'box',
              'x': array(['LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG',
                          'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([4985, 683, 3067, 534, 2257, 709, 1166, 996, 2561, 610, 2251, 446, 902,
                          2783, 1095, 1287], dtype=object),
              'y0': ' ',
              'yaxis': 'y'}],
    'layout': {'boxmode': 'overlay',
               'legend': {'title': {'text': 'Algorithm'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Module Sizes'},
               'xaxis': {'anchor': 'y',
                         'categoryarray': [WGCNA, LBG],
                         'categoryorder': 'array',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'Algorithm'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Size'}}}
})

In [26]:
px.box(svm_results, x = 'Algorithm', y = 'BSR', color = 'Algorithm', points= 'all', title = 'Classification BSR')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'hovertemplate': 'Algorithm=%{x}<br>BSR=%{y}<extra></extra>',
              'legendgroup': 'WGCNA',
              'marker': {'color': '#636efa'},
              'name': 'WGCNA',
              'notched': False,
              'offsetgroup': 'WGCNA',
              'orientation': 'v',
              'showlegend': True,
              'type': 'box',
              'x': array(['WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA',
                          'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA'],
                         dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([1.   , 0.5  , 1.   , 1.   , 1.   , 0.875, 1.   , 0.5  , 0.875, 0.875,
                          0.375, 0.625, 0.875, 0.75 , 0.875, 0.875]),
              'y0': ' ',
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'boxpoints': 'all',
              'hovertemplate': 'Algorithm=%{x}<br>BSR=%{y}<extra></extra>',
              'legendgroup': 'LBG',
              'marker': {'color': '#EF553B'},
              'name': 'LBG',
              'notched': False,
              'offsetgroup': 'LBG',
              'orientation': 'v',
              'showlegend': True,
              'type': 'box',
              'x': array(['LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG',
                          'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([1.   , 0.75 , 0.875, 0.875, 1.   , 0.75 , 0.625, 0.75 , 1.   , 1.   ,
                          0.75 , 0.375, 1.   , 1.   , 1.   , 1.   ]),
              'y0': ' ',
              'yaxis': 'y'}],
    'layout': {'boxmode': 'overlay',
               'legend': {'title': {'text': 'Algorithm'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Classification BSR'},
               'xaxis': {'anchor': 'y',
                         'categoryarray': [WGCNA, LBG],
                         'categoryorder': 'array',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'Algorithm'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'BSR'}}}
})

In [27]:
px.box(go_results, x = 'Algorithm', y = 'GO Significance', color = 'Algorithm', points= 'all', title = 'GO Significance')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'hovertemplate': 'Algorithm=%{x}<br>GO Significance=%{y}<extra></extra>',
              'legendgroup': 'WGCNA',
              'marker': {'color': '#636efa'},
              'name': 'WGCNA',
              'notched': False,
              'offsetgroup': 'WGCNA',
              'orientation': 'v',
              'showlegend': True,
              'type': 'box',
              'x': array(['WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA',
                          'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA', 'WGCNA'],
                         dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([5823.55234493,  125.31405881, 4401.76264818,  162.89685568,
                          2175.59610386,   63.29043659,  425.06312413,  352.27274197,
                          2983.19192221,    0.        ,    0.        ,  221.51634872,
                           554.80292779,   11.47700455,  653.9681989 ,  409.85493914]),
              'y0': ' ',
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'boxpoints': 'all',
              'hovertemplate': 'Algorithm=%{x}<br>GO Significance=%{y}<extra></extra>',
              'legendgroup': 'LBG',
              'marker': {'color': '#EF553B'},
              'name': 'LBG',
              'notched': False,
              'offsetgroup': 'LBG',
              'orientation': 'v',
              'showlegend': True,
              'type': 'box',
              'x': array(['LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG',
                          'LBG', 'LBG', 'LBG', 'LBG', 'LBG', 'LBG'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([6.54887441e+03, 3.56106249e+02, 3.44220452e+03, 1.53380801e+00,
                          1.75740471e+03, 3.91121334e+01, 6.78389203e+02, 8.42686066e+01,
                          6.82259923e+03, 1.42415951e+01, 5.86225261e+01, 7.71217023e+01,
                          8.36653034e+02, 4.60742418e+03, 7.94569686e+02, 1.69841839e+02]),
              'y0': ' ',
              'yaxis': 'y'}],
    'layout': {'boxmode': 'overlay',
               'legend': {'title': {'text': 'Algorithm'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'GO Significance'},
               'xaxis': {'anchor': 'y',
                         'categoryarray': [WGCNA, LBG],
                         'categoryorder': 'array',
                         'domain': [0.0, 1.0],
                         'title': {'text': 'Algorithm'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'GO Significance'}}}
})

In [25]:
svm_better['Statistic'] = 'Classification BSR'
go_better['Statistic'] = 'GO Significance'
px.bar(pd.concat([svm_better, go_better]), y = 'Relative Gain', x = 'Statistic', 
       title = f'Relative gain for LBG modules over WGNCA modules <br>Module representative: {center_method} <br>Center dimension: {center_dimension}, Data dimension: {data_dimension}',
       barmode = 'group')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Statistic=%{x}<br>Relative Gain=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Classification BSR', 'GO Significance'], dtype=object),
              'xaxis': 'x',
              'y': array([0.05768417, 0.43150545]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'group',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': ('Relative Gain for LBG Modules ' ... 'mension: 1, >data_dimension: 1')},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Statistic'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Relative Gain'}}}
})